# Introduction
This is a semi-manual tool to crawl and extract information through large amount of urls. There are parts you will need to manual input numbers based on the volumn of urls. The tool goes by a batch run format with threads to ensure crawl efficency.

This tool also includes writing into the AWS Dynamo database. 

In [1]:
#install all needed packages
import datetime
import MySQLdb
from langdetect import detect
import requests
from bs4 import UnicodeDammit
from langdetect import detect
from newspaper import Article
from langdetect.lang_detect_exception import LangDetectException
import pandas as pd
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree
from forex_python.converter import CurrencyCodes
import threading
import time
import re
import datetime
import json
import boto3
import decimal

## Global dictionary: Contractions
Stored for text cleaning purpose

In [2]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

# Function: get_contious_chunks

NLTK Based rough NER tool for reference

Input: text

Input type: string

Output: a list of NERs

Output type: list


In [3]:
def get_continuous_chunks(text):
    c = CurrencyCodes()
    Clean_NER = []
    NER = []
    chunked = ne_chunk(pos_tag(word_tokenize(text)))
    prev = None
    continuous_chunk = []
    current_chunk = []
    for i in chunked:
        if type(i) == Tree:
            current_chunk.append(" ".join([token for token, pos in i.leaves()]))
            current_chunk = list(set(current_chunk)) #remove duplicates
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
            else:
                continue
    NER = list(set(continuous_chunk)) #remove duplicates 
    for i in NER:
        if c.get_symbol(i[:3]) is None:
            Clean_NER.append(i)
        
        NER = Clean_NER 

    return NER  

# Function: clean_text
A rough clean for text mining purposes, *note there's a more established version in the actual keyword tool script. This one is only for reference*

Input: text

Input type: string

Output: cleaned text

Output type: string

In [4]:
def clean_text(text):   
    #c = CurrencyCodes()
    #for i in text:
    #    if c.get_symbol(i[:3]) is None:
    #        i = "" #clear currency
    for contraction in contractions.keys():
        text = text.replace(contraction,contractions[contraction]) #all contraction to formal forms
    cxo = ['CAO','CAIO','CIO','CFO','CTO','CEO','COO','CAE','CBO','CBDO','CCO','CDO','CEngO','CHRO','CMO','CNO','CPO','CRO','CVO']
    for term in cxo:
        text = text.replace(term, "") #clean text with CXO    
    text = re.sub(r'http\S+', '',text, flags = re.IGNORECASE) # remove in text urls with http
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE) # remove in text https
    text = re.sub(r'www.\S+', '',text, flags = re.IGNORECASE) #remove www urls
    text = re.sub(r'\bmobile:\s*\b', '',text, flags = re.IGNORECASE) #remove mobile
    text = re.sub(r'\bphone:\s*\b', '',text, flags = re.IGNORECASE) #remove phone
    text = re.sub(r'\be*-*mail:\s*\b', '',text, flags = re.IGNORECASE) #remove email
    text = re.sub(r'\S*tel:S*', '',text, flags = re.IGNORECASE) # remove number
    text = re.sub(r'\S*fax:S*', '',text, flags = re.IGNORECASE) # remove fax
    text = re.sub(r'\baddress:\s*\b', '',text, flags = re.IGNORECASE) #remove address
    text = re.sub(r'\bwebsite:\s*\b', '',text, flags = re.IGNORECASE) #remove image
    text = re.sub(r'\bimage:\s*\b', '',text, flags = re.IGNORECASE) #remove image
    text = re.sub(r'\S*@\S*\s?','',text, flags=re.MULTILINE) #remove in text email
    text = re.sub(r'\S*\.com\S*\s?','',text, flags=re.MULTILINE) #remove .com urls
    text = re.sub(r'\S*\.uk\S*\s?','',text, flags=re.MULTILINE) #remove .uk urls
    text = text.replace('+',"")
    text = text.replace('-',"")
    text = text.replace('(',"")
    text = text.replace(')',"")
    text = text.replace('tel',"")
    text = text.replace('Tel',"")
    text = text.replace('TEL',"")
    text = text.replace('CET',"")
    text = text.replace('%',"")
    text = text.replace('$',"")
    text = text.replace('%',"")
    text = text.replace('percent',"")
    text = text.replace('£',"")
    text = ''.join([i for i in text if not i.isdigit()])
    return text

# Function: get_tier_3 (Manual script modification required for adjustments)
Get relevant PB tier 3 database information. No input required. 

*Function returns only two lists of urls and time stamps*

Manual modificaiton areas:
date range

please see in function notes for details

In [67]:
def get_tier_3():
    """
    import all urls into a list
    """
    import db_credit as dbc
    db = MySQLdb.connect(host= dbc.host,  
                         user= dbc.user,       
                         passwd= dbc.passwd,
                         port = dbc.port,
                         db= dbc.db)
     
    # Create a Cursor object to execute queries.
    cur = db.cursor()
     
    # Select data from table using SQL query.
    #***NOTE: For date range modifications, user must manual change date range in the same formate below***

    cur.execute(
                """
                select a.url from tier_3_output.article a
                where a.timestamp between '2017-01-26 00:00:00' and '2017-01-26 23:00:00'
                """)    
    url_list = []            
    urls = list(cur.fetchall())
    
    for url in urls:
        url_list.append(url[0])
        
    # Select data from table using SQL query.    
    #***NOTE: For date range modifications, user must manual change date range in the same formate below***
    cur.execute(
                """
                select Date(a.timestamp) from tier_3_output.article a
                where a.timestamp between '2017-01-26 00:00:00' and '2017-01-26 23:00:00'
                """)    
    time_list = []            
    times = list(cur.fetchall())
    
    for time in times:
        time_list.append(time[0])    
        
    print ("DB info gathering completed")
    return url_list, time_list

# Function: get_url_info
This is one of the cores of this scrip. The funciton crawl title and text from the news article

Input: url

Input type: string

Output: title, text, url

Output type: string(s)

In [6]:
def get_url_info(url):
    try:
        r = requests.head(url) 
        if r.status_code < 400: # if loads
            article = Article(url)
            article.download()
            article.parse()
            if detect(article.title) == 'en': #English only
                if len(article.text)>400: #filter out permission request
                    title = UnicodeDammit(article.title).unicode_markup
                    text = UnicodeDammit(article.text).unicode_markup
                    #text = clean_text(text)
                    test_url= url
                    #NER = get_continuous_chunks(text)
                    #timestamp = article.publish_date
                    #print("success! ", datetime.datetime.now().time())
                    return title, text, test_url#, NER #timestamp
    except Exception as e:
        print("fail ", datetime.datetime.now().time())

# Function: append_url_info
*call back to get_url_info, the target for threading calls, append each url crawl result into lists*

Input: url

Input type: string

Output: title_list, text_raw_list, url_list

Output: list(s)

In [7]:
def append_url_info(url):
    try:
        title, text, test_url = get_url_info(url)
        if 'Please Sign In' not in text:
            title_list.append(title)
            text_raw_list.append(text)
            test_urls.append(url)
            #NER_list.append(NER)
            #time_list.append(timestamp) 
        #elif not (NER[0] == 'Please Sign' or NER[0] == 'Javascript'):
            #title_list.append(title)
            #text_raw_list.append(text)
            #test_urls.append(url)
            #NER_list.append(NER)
            #time_list.append(timestamp) 
            print (len(test_urls))
        return title_list, text_raw_list, test_urls#, NER_list#, time_list
    except (TypeError):
        pass

# Non-funtion scripts starts here
The flowing chunck get DB information and call back to get_tier_3

Run the below block to get all urls and timestamps from the time you hard-coded 

In [69]:
url_list, date_list = get_tier_3()
unique_date = set(date_list)    
    
date_url_dict = {}
for date in unique_date:
    date_url_dict[date] = []   
    
for date_key in date_url_dict.keys():
    for i in range(len(date_list)):
        if date_list[i] == date_key:
            date_url_dict[date_key].append(url_list[i])
            
url_combine = []    
for date_key in date_url_dict.keys():
    urls = date_url_dict[date_key]
    for url in urls:
        url_combine.append(url)
        
times = date_list            
times = [str(i) for i in times]
diction = {}
for i in range(len(url_list)):
    diction[url_list[i]] = times[i]        

DB info gathering completed


The print tells you the size of data, for you to get an idea of how to slice that into batches. We reccomend slicing for any volun larger than 2k

In [70]:
print(len(url_combine))

0


the list "combine" is a sliced list of the original "url_combine" in case the volumn is > 10k, below example is for slicing data >10k

In [68]:
combine = url_combine[:10000] #update this for data >10k to for example [10000:20000], and [20000:30000] after each run is finished
print(len(combine))

1820


Further slice into batches then combine

In [50]:
batches = []
#batches.append(combine) #use this and comment out the rest for data <1000
batches.append(combine[:1000])
batches.append(combine[1000:2000])
batches.append(combine[2000:3000])
batches.append(combine[3000:4000])
batches.append(combine[4000:5000])
batches.append(combine[5000:6000])
batches.append(combine[6000:7000])
batches.append(combine[7000:8000])
batches.append(combine[8000:9000])
batches.append(combine[9000:10000])

# DYNAMO DB writing scripts starts here
For each batch int he batch list complibed above, the script writes data into the databse, for each batch it will return the total number of rows inserted.

In [45]:
for batch in batches:
    title_list, text_raw_list, test_urls, NER_list, time_list = [], [], [], [], []

    threads = [threading.Thread(target=append_url_info, args=(url,)) for url in batch] #url_combined in date sample method

    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

    text_clean_list = [clean_text(text) for text in text_raw_list]

    NER_list = [get_continuous_chunks(title) for title in title_list]

    for url in test_urls:
        if url in diction.keys():
            time_list.append(diction[url])

    summary = {'url': test_urls, 'title': title_list, 'text_raw': text_raw_list, 'text_clean':text_clean_list, 'NER': NER_list, 'timestamp': time_list}
    summary_df = pd.DataFrame(summary)

    #clean data
    import collections
    a = summary_df['title'].tolist()
    dup_title = [item for item, count in collections.Counter(a).items() if count >1]
    for title in summary_df['title']:
        if title in dup_title:
            summary_df = summary_df[summary_df['title'] != title]

    #start writing data into DB        
    obj = json.loads(summary_df.to_json(orient='records'))

    dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

    table = dynamodb.Table('url_text')

    for record in obj:
        NER = record['NER']
        text_clean = record['text_clean']
        text_raw = record['text_raw']
        timestamp = record['timestamp']
        title = record['title']
        url = record['url']

        table.put_item(
            Item={
                    'NER': NER,
                    'text_clean': text_clean,
                    'text_raw': text_raw,
                    'timestamp': timestamp,
                    'title': title,
                    'url': url,
                }
            )
    print ('batch-run finished: ',datetime.datetime.now().time())

1
fail  04:59:50.902274
You must `download()` an article first!
fail  04:59:54.188131
You must `download()` an article first!
fail  You must `download()` an article first!04:59:54.287196
You must `download()` an article first!You must `download()` an article first!
fail 

fail  fail  04:59:54.295203 04:59:54.299704

04:59:54.308710
You must `download()` an article first!
You must `download()` an article first!fail You must `download()` an article first!You must `download()` an article first!You must `download()` an article first!You must `download()` an article first!You must `download()` an article first!You must `download()` an article first! 






04:59:54.380759fail fail fail fail fail fail fail 
       04:59:54.43680904:59:54.41629504:59:54.39076504:59:54.41929804:59:54.42730304:59:54.42580204:59:54.439310






You must `download()` an article first!
fail  04:59:55.988369
You must `download()` an article first!
fail  04:59:56.112486
You must `download()` an article first!
fail  

Article `download()` failed with 403 Client Error: Forbidden for url: https://www.bizjournals.com/portland/news/2017/02/07/bill-to-boost-small-scale-renewables-in-oregon.html?ana=RSS%26s%3Darticle_search&utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+industry_5+%28Industry+Energy+%26+the+Environment%29 on URL http://feeds.bizjournals.com/~r/industry_5/~3/8-LyZzgP3PY/bill-to-boost-small-scale-renewables-in-oregon.html
fail  05:00:07.136361
You must `download()` an article first!Article `download()` failed with 403 Client Error: Forbidden for url: https://www.bizjournals.com/newyork/news/2017/02/07/meredith-taking-another-run-at-buying-time-inc.html?ana=RSS%26s%3Darticle_search&utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%3A+industry_18+%28Industry+Media+%26+Marketing%29 on URL http://feeds.bizjournals.com/~r/industry_18/~3/UNscodkA9Jo/meredith-taking-another-run-at-buying-time-inc.html

Article `download()` failed with 403 Client Error: Forbidden for url: https://